<a href="https://colab.research.google.com/github/Akhil-blanc/Implementation-FaceNet-SiameseNet/blob/main/Copy_of_Facenetchecks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset,BatchSampler
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
cuda = torch.cuda.is_available()

In [2]:
data = torchvision.datasets.CelebA(root = "/content", download = True)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [3]:
data[3]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F97FAA442B0>,
 tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1]))

In [4]:
mat = data[5][0]

In [5]:
print(mat)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F97F87715E0>


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

eps = 1e-8 

def _pairwise_distances(embeddings, squared=False):
    """Compute the 2D matrix of distances between all the embeddings.

    Args:
        embeddings: tensor of shape (batch_size, embed_dim)
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.

    Returns:
        pairwise_distances: tensor of shape (batch_size, batch_size)
    """
    dot_product = torch.matmul(embeddings, embeddings.t())

    # Get squared L2 norm for each embedding. We can just take the diagonal of `dot_product`.
    # This also provides more numerical stability (the diagonal of the result will be exactly 0).
    # shape (batch_size,)
    square_norm = torch.diag(dot_product)

    # Compute the pairwise distance matrix as we have:
    # ||a - b||^2 = ||a||^2  - 2  + ||b||^2
    # shape (batch_size, batch_size)
    distances = square_norm.unsqueeze(0) - 2.0 * dot_product + square_norm.unsqueeze(1)

    # Because of computation errors, some distances might be negative so we put everything >= 0.0
    distances[distances < 0] = 0

    if not squared:
        # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
        # we need to add a small epsilon where distances == 0.0
        mask = distances.eq(0).float()
        distances = distances + mask * 1e-16

        distances = (1.0 -mask) * torch.sqrt(distances)

    return distances

def _get_triplet_mask(labels):
    """Return a 3D mask where mask[a, p, n] is True iff the triplet (a, p, n) is valid.
    A triplet (i, j, k) is valid if:
        - i, j, k are distinct
        - labels[i] == labels[j] and labels[i] != labels[k]
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    """
    # Check that i, j and k are distinct
    indices_equal = torch.eye(labels.size(0), device=labels.device).bool()
    indices_not_equal = ~indices_equal
    i_not_equal_j = indices_not_equal.unsqueeze(2)
    i_not_equal_k = indices_not_equal.unsqueeze(1)
    j_not_equal_k = indices_not_equal.unsqueeze(0)

    distinct_indices = (i_not_equal_j & i_not_equal_k) & j_not_equal_k


    label_equal = labels.unsqueeze(0) == labels.unsqueeze(1)
    i_equal_j = label_equal.unsqueeze(2)
    i_equal_k = label_equal.unsqueeze(1)

    valid_labels = ~i_equal_k & i_equal_j

    return valid_labels & distinct_indices


def _get_anchor_positive_triplet_mask(labels):
    """Return a 2D mask where mask[a, p] is True iff a and p are distinct and have same label.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check that i and j are distinct
    indices_equal = torch.eye(labels.size(0), device=labels.device).bool()
    indices_not_equal = ~indices_equal

    # Check if labels[i] == labels[j]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = labels.unsqueeze(0) == labels.unsqueeze(1)

    return labels_equal & indices_not_equal


def _get_anchor_negative_triplet_mask(labels):
    """Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check if labels[i] != labels[k]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)

    return ~(labels.unsqueeze(0) == labels.unsqueeze(1))

In [7]:
def batch_hard_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.

    For each anchor, we get the hardest positive and hardest negative to form a triplet.

    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.

    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist =_pairwise_distances(embeddings)

    # For each anchor, get the hardest positive
    # First, we need to get a mask for every valid positive (they should have same label)
    mask_anchor_positive = _get_anchor_positive_triplet_mask(labels).float()

    # We put to 0 any element where (a, p) is not valid (valid if a != p and label(a) == label(p))
    anchor_positive_dist = mask_anchor_positive * pairwise_dist

    # shape (batch_size, 1)
    hardest_positive_dist, _ = anchor_positive_dist.max(1, keepdim=True)

    # For each anchor, get the hardest negative
    # First, we need to get a mask for every valid negative (they should have different labels)
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels).float()

    # We add the maximum value in each row to the invalid negatives (label(a) == label(n))
    max_anchor_negative_dist, _ = pairwise_dist.max(1, keepdim=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)

    # shape (batch_size,)
    hardest_negative_dist, _ = anchor_negative_dist.min(1, keepdim=True)

    # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
    tl = anchor_positive_dist - anchor_negative_dist + margin
    tl = F.relu(tl)
    tl=torch.where(tl<margin,tl,0)
    if embeddings.is_cuda:
      tl=tl.cuda()
    print(tl.shape)
    triplet_loss = tl.mean()

    return triplet_loss

In [8]:
t = torch.randn(192,55,55)
model2 = nn.MaxPool2d(3, stride=2, padding=1)
print(model2(t).shape)

torch.Size([192, 28, 28])


In [9]:
class NN1(nn.Module):
    def __init__(self):
        super(NN1, self).__init__()
        self.convlayers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(3, stride=2),
            nn.LocalResponseNorm(5),
            nn.Conv2d(64, 64, kernel_size=1, stride=1),
            nn.Conv2d(64, 192, kernel_size=3, stride=1),
            nn.ReLU(inplace=False),
            nn.LocalResponseNorm(5),
            nn.MaxPool2d(3, stride=2),
            nn.Conv2d(192, 192, kernel_size=1, stride=1),
            nn.Conv2d(192, 384, kernel_size=3, stride=1,padding="same"),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(3, stride=2,padding=1),
            nn.Conv2d(384, 384, kernel_size=1, stride=1,padding="same"),
            nn.Conv2d(384, 256, kernel_size=3, stride=1,padding="same"),
            nn.ReLU(inplace=False),
            nn.Conv2d(256, 256, kernel_size=1, stride=1,padding="same"),
            nn.Conv2d(256, 256, kernel_size=3, stride=1,padding="same"),
            nn.ReLU(inplace=False),
            nn.Conv2d(256, 256, kernel_size=1, stride=1,padding="same"),
            nn.Conv2d(256, 256, kernel_size=3, stride=1,padding="same"),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(3, stride=2,padding=1),
        )
        self.fc = nn.Sequential(
            nn.Linear(12544, 4096),
            nn.Linear(4096, 4096),
            nn.Linear(4096, 128)
        )

    def forward(self, x):
        conv_output = self.convlayers(x)
        flattened_output = nn.Flatten(1)(conv_output)
        fc_output = self.fc(flattened_output)
        norm_output = nn.functional.normalize(fc_output)
        output = nn.Flatten(1)(norm_output)
        return output

In [23]:

model = NN1()
if cuda:
    model.cuda()
optimizer = optim.Adagrad(model.parameters(), lr = 0.005 )


In [12]:
class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, dataset, n_classes, n_samples):
        loader = DataLoader(dataset)
        self.labels_list = []
        for _, label in loader:
            self.labels_list.append(label)
        self.labels = torch.LongTensor(self.labels_list)
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.dataset = dataset
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < len(self.dataset):
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return len(self.dataset) // self.batch_size

In [15]:
train_data = torchvision.datasets.CelebA(root = "/content", split = "train",target_type='identity', download = True, transform = transforms.Compose([transforms.Resize((220, 220)), transforms.PILToTensor(), transforms.ConvertImageDtype(dtype = torch.float32)]))
train_batch_sampler = BalancedBatchSampler(train_data, n_classes=10, n_samples=25)

Files already downloaded and verified


In [19]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = DataLoader(train_data, batch_sampler=train_batch_sampler, **kwargs)



In [ ]:
# labels=train_data.attr
# print(labels)
# list_=[]
# for i,name in enumerate(train_data.class_to_idx):
#   list_.append(torch.numel(torch.nonzero(torch.where(labels==i,labels,0))))
# listt=torch.tensor(list_)
# print(torch.numel(torch.nonzero(torch.where(listt>2,listt,0))))
# for idx,batch in enumerate(train_loader):
#   if idx==1:
#     print(batch[1].shape)
# print("Training Dataloader Created!!")

In [17]:
from torchvision import models
from torchsummary import summary

summary(model,(3,220,220))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 107, 107]           9,472
              ReLU-2         [-1, 64, 107, 107]               0
         MaxPool2d-3           [-1, 64, 53, 53]               0
 LocalResponseNorm-4           [-1, 64, 53, 53]               0
            Conv2d-5           [-1, 64, 53, 53]           4,160
            Conv2d-6          [-1, 192, 51, 51]         110,784
              ReLU-7          [-1, 192, 51, 51]               0
 LocalResponseNorm-8          [-1, 192, 51, 51]               0
         MaxPool2d-9          [-1, 192, 25, 25]               0
           Conv2d-10          [-1, 192, 25, 25]          37,056
           Conv2d-11          [-1, 384, 25, 25]         663,936
             ReLU-12          [-1, 384, 25, 25]               0
        MaxPool2d-13          [-1, 384, 13, 13]               0
           Conv2d-14          [-1, 384,

In [24]:
epochs = 1
for j in range(epochs):
  counter = 0
  for batch_idx, (data, target) in enumerate(train_loader):
    target = target if len(target) > 0 else None
    if not type(data) in (tuple, list):
        data = (data,)
    if cuda:
        data = tuple(d.cuda() for d in data)
        if target is not None:
            target = target.cuda()
    embeddings = model(*data)
    optimizer.zero_grad()
    currloss = batch_hard_triplet_loss(target,embeddings,0.2)
    currloss.backward()
    optimizer.step()
    counter+=1
    print(epochs," ",counter,"",currloss.item())
  


torch.Size([187, 187])
1   1  0.1986234337091446
torch.Size([203, 203])
1   2  0.17929565906524658
torch.Size([189, 189])
1   3  0.1860792487859726
torch.Size([217, 217])
1   4  0.04988572746515274
torch.Size([200, 200])
1   5  0.14504098892211914
torch.Size([213, 213])
1   6  0.11561812460422516
torch.Size([195, 195])
1   7  0.0704045444726944
torch.Size([159, 159])
1   8  0.1060238853096962
torch.Size([140, 140])
1   9  0.03345661237835884
torch.Size([212, 212])
1   10  0.014690150506794453
torch.Size([177, 177])
1   11  0.011037030257284641
torch.Size([227, 227])
1   12  0.008549155667424202
torch.Size([202, 202])
1   13  0.002627317328006029
torch.Size([175, 175])
1   14  0.0017786998068913817
torch.Size([189, 189])
1   15  0.0016604436095803976
torch.Size([182, 182])
1   16  0.0017772880382835865
torch.Size([161, 161])
1   17  0.000970776891335845
torch.Size([167, 167])
1   18  0.0006393833318725228
torch.Size([158, 158])
1   19  0.0006713335169479251
torch.Size([164, 164])
1   20